# Preliminaries

In [1]:
# numpy and pandas imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# ML Libraries for scikit
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn import tree
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,\
                            precision_recall_curve,precision_score,recall_score,roc_auc_score,roc_curve,\
                            matthews_corrcoef, f1_score, make_scorer, auc
from scipy.stats import skew

#  Recursive Feature Elimination
from sklearn.feature_selection import RFE
        
# Python imports
from math import log, sqrt
import re
import collections
import warnings

# decision tree visualization related imports
import pydotplus
import graphviz

from IPython.display import Image

warnings.filterwarnings('ignore')

pd.options.display.max_columns = 400

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Loading Data set

In [3]:
df_original = pd.read_csv('UCI_Credit_Card.csv',index_col='ID')

In [4]:
df = df_original.copy()

## About DataSet and Some dataset column naming standardizations
+ Column naming standardization (Replacing pay_0 with pay_1 to make it standard with other columns)
+ lower case names for columns (referring to them using column name would make it easy)
+ Dataset - domain information and data type information, interpretations

We lowercase the column name, and rename the column names when required, In particular, remarkably this dataset misses a colum  PAY_1. In the analysis here below we assume that PAY_0 is actually pay_1, to be consider the repayment of the month prior to the month where we calculate the defaulting (which is October 2005, in this particular dataset)

In [5]:
df.columns = [x.lower() for x in df.columns]
df = df.rename(columns={"pay_0": "pay_1","default payment next month":"default_pay"})
cols = df.columns.values
col_names = [str(s) for s in cols]
# col_names

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 1 to 30000
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   limit_bal                   30000 non-null  float64
 1   sex                         30000 non-null  int64  
 2   education                   30000 non-null  int64  
 3   marriage                    30000 non-null  int64  
 4   age                         30000 non-null  int64  
 5   pay_1                       30000 non-null  int64  
 6   pay_2                       30000 non-null  int64  
 7   pay_3                       30000 non-null  int64  
 8   pay_4                       30000 non-null  int64  
 9   pay_5                       30000 non-null  int64  
 10  pay_6                       30000 non-null  int64  
 11  bill_amt1                   30000 non-null  float64
 12  bill_amt2                   30000 non-null  float64
 13  bill_amt3                   30000 no

<span style="color:black; font-family: 'calibri'; font-size: 1.2em;">As you can see from the above information, the dataset does not look to have missing values.</b>

In [7]:
print("Explanatory variables:  {}".format(len(df.columns)-1))
print("Number of Observations: {}".format(df.shape[0]))

df['default_pay'] = df['default_pay'].astype('category')

# create a 'target' column for our own convenience
print("Target variable:       '{}' -> '{}'".format('default payment next month', 'default_pay'))

Explanatory variables:  23
Number of Observations: 30000


KeyError: 'default_pay'

### DataSet description
<span style="color:black; font-family: 'calibri'; font-size: 1.2em;">This study leverages following 
+ <b>Dependent Variable</b>  : default payment (Yes = 1, No = 0), as the response variable
+ <b>Independent Variable</b>: 23 variables as explanatory variables and they are numeric. These are as follows : 
    
```
   ----------------------------------------------------------------------------------------------------------------------
   Name                   Numeric / Categorical          Explantion        
   ----------------------------------------------------------------------------------------------------------------------
   limit_bal              Numeric                        Amount of the given credit (NT dollar): 
                                                         it includes both the individual consumer credit 
                                                         and his/her family (supplementary) credit.
   
   sex                    Categorical                    Gender 
                                                         (1 = male; 2 = female)
   
   education              Categorical                  Education
                                                       (1 = graduate school; 2 = university; 3 = high school; 4 = others)
   
   marriage               Categorical                    Marital status 
                                                         (1 = married; 2 = single; 3 = others)
   
   age                    Numeric                        Age (years)
   
   pay_0 - pay_6          Numeric                        History of past payment. Past monthly payment records 
                                                         From April to September, 2005 as follows:
                        
                                                         pay_0 = the repayment status in September, 2005
                                                         pay_2 = the repayment status in August, 2005
                                                         ...
                                                         pay_6 = the repayment status in April, 2005 
                        
                                                         The measurement scale for the repayment status is: 
                                                         -1 = pay duly; 
                                                         1 = payment delay for one month 
                                                         2 = payment delay for two months
                                                         ...
                                                         8 = payment delay for eight months 
                                                         9 = payment delay for nine months and above
                        
   bill_amt1-bill_amt5    Numeric                        Amount of bill statement (NT dollar). 
                                                         bill_amt1 = amount of bill statement in September, 2005 
                                                         bill_amt2 = amount of bill statement in August, 2005
                                                         ...
                                                         bill_amt6= amount of bill statement in April, 2005 
                        
   pay_amt1-pay_amt6      Numeric                        Amount of previous payment (NT dollar)
                                                         pay_amt1 = amount paid in September, 2005
                                                         pay_amt2 = amount paid in August, 2005
                                                         ...
                                                         pay_amt6 = amount paid in April, 2005 
   ----------------------------------------------------------------------------------------------------------------------
```
</span>

In [ ]:
df.head(5)

***

## Some initializations and reusable methods block

Collect all the mertics at one place for all the models used during training. This will help in comarison of these metrics.

In [ ]:
# commonly used variables
pay_status_columns = []
for x in range(1,7): pay_status_columns.append("pay_"+ str(x))
    
bill_amt_columns = []
for x in range(1,7): bill_amt_columns.append("bill_amt"+ str(x))
    
pay_amt_columns = []
for x in range(1,7): pay_amt_columns.append("pay_amt"+ str(x))


# initialization block for ML
perf_metrics = ['Model','Model_Desc','TP','FP','FN','TN','AUC',\
                'Accuracy_Score','Precision','Recall',\
                'F1_Score', 'Matt_Coeff']
df_all_models = pd.DataFrame(columns=perf_metrics)
perf_metric_per_model = {}

In [ ]:
# This is only required during model building exercise
# for model evaluation
# Get the performance metrics for the models
def save_performance_metrics(model, model_name, model_dsc, X_test, y_test, X_train=None, y_train=None):
    # nested inner method for adding performance metric
    def add_perf_metric_to_compare(perf_metric_per_model):
        global df_all_models
        model_name = perf_metric_per_model['Model']
        # if model data already exists then drop those rows
        if df_all_models[df_all_models['Model'] == model_name].shape[0] >= 1:
            ind = df_all_models[df_all_models['Model']== model_name].index
            df_all_models.drop(ind, inplace=True)
        s1 = pd.Series(perf_metric_per_model, perf_metric_per_model.keys())
        df_all_models = df_all_models.append(s1, ignore_index=True)    
    
    perf_metric_per_model = {}
#     print "Evaluating performance for {} description is {}"\
#                                         .format(model_name,model_description)
    perf_metric_per_model['Model'] = model_name
    perf_metric_per_model['Model_Desc'] = model_dsc
    y_pred = model.predict(X_test)
    conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    tn, fp, fn, tp = conf_mat.ravel()
    perf_metric_per_model['TP'], perf_metric_per_model['FP'] = tp, fp
    perf_metric_per_model['FN'], perf_metric_per_model['TN'] = fn, tn
    perf_metric_per_model['AP'], perf_metric_per_model['AN'] = tp+fn, fp+tn
    perf_metric_per_model['AUC'] = roc_auc_score(y_test, y_pred)
    perf_metric_per_model['F1_Score'] = f1_score(y_test, y_pred)
    perf_metric_per_model['Accuracy_Score'] = accuracy_score(y_test, y_pred)
    perf_metric_per_model['Precision'] = precision_score(y_test, y_pred)
    perf_metric_per_model['Recall'] = recall_score(y_test, y_pred) 
    perf_metric_per_model['Train_score'] = model.score(X_train, y_train)    
    perf_metric_per_model['Test_score'] = model.score(X_test, y_test)    
    perf_metric_per_model['Matt_Coeff'] = matthews_corrcoef(y_test, y_pred)
    s2 = pd.Series(perf_metric_per_model, index=perf_metric_per_model.keys())
    df_metric = pd.DataFrame(data=[s2], columns=perf_metrics)
    add_perf_metric_to_compare(perf_metric_per_model)
    return df_metric
    
# plot feature importances
# not applicable to Logistic Regression
def plot_feature_importance(model_name, importances, columns):
    # feature importances
    plt.figure(figsize=(12,6))
    indices = np.argsort(importances)[::-1]
    indices = indices[:10]
    plt.title('Top 10 Feature Importances by {}'.format(model_name))
    plt.barh(range(len(indices)), importances[indices], color='b', align='center')
    plt.yticks(range(len(indices)), columns[indices])
    plt.xlabel('Relative Importance')
    plt.show()

# plot ROC and Precision, Recall curve
def plot_roc_and_precision(model_name, model, X_test, y_test):
    # calculate the fpr and tpr for all thresholds of the classification
    probs = model.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = roc_curve(y_test, preds)
    roc_auc = auc(fpr, tpr)
    
    # AUC under ROC Curve 
    plt.figure(figsize=(12,6))
    plt.subplot(121)
    plt.title('Receiver Operating Characteristic for {}'.format(model_name))
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('TPR (Recall or Sensitivity)')
    plt.xlabel('FPR (1-Specificity)')
    
    # precision and recall curve
    plt.subplot(122)
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    plt.step(recall, precision, color='b', alpha=0.1,
         where='post')
    plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall curve for {}'.format(model_name))
    
    plt.tight_layout(pad=4.0, w_pad=0.5, h_pad=1.0)    
    plt.show()

metrics = pd.DataFrame(index=['accuracy','precision','recall','auc_roc_score'],columns=['LogR','DTree','RF'])
# metrics    
def CMatrix(CM,labels=['pay','default']):
    df1 = pd.DataFrame(data=CM,index=labels,columns=labels)
    df1.index.name = 'Actual'
    df1.columns.name = 'Predicted'
    df1.loc['Total'] = df1.sum()
    df1['Total'] = df1.sum(axis=1)
    return df1

# EDA

## Information related to Credit Limit, Payment Status, Bill Amount and Bill Payment
We will look at how payment status, bill amount and bill payment columns look like.

In [ ]:
total_cnt = df.shape[0]
print 'Class Statistics on entire data {}:'.format(total_cnt)
cls_cnt = df['default_pay'].value_counts()
cls_stats = pd.DataFrame ({"Count": [cls_cnt[0], cls_cnt[1]],\
                        "Percent Split" : [((cls_cnt[0] * 100.0) /total_cnt), ((cls_cnt[1] * 100.0) /total_cnt)]},
                         index=["Not Default", "Default"])
display(cls_stats)
plt.figure(figsize=(5,5))
ax = cls_cnt.plot(kind='bar')
ax.set_xlabel("Default Status")
ax.set_ylabel("No. of Instances")
ax.set_xticklabels(['Not Default', 'Default'])
#     for p in ax.patches:
#         ax.annotate('{}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+300))
plt.tight_layout()
plt.show()

## Information related to Credit Limit, Payment Status, Bill Amount and Bill Payment
We will look at how payment status, bill amount and bill payment columns look like.

In [ ]:
df[pay_status_columns].head(10)

In [ ]:
print 'Paid Amount Columns'
display(df[pay_amt_columns].head(10))
print ('==')*10
print '\nPaid Amount Columns Statistics'
display(df[pay_amt_columns].describe())

In [ ]:
print 'Bill Amount Columns'
display(df[bill_amt_columns].head(10))
print ('==')*10
print '\nBill Amount Columns Statistics'
display(df[bill_amt_columns].describe())

In [ ]:
df['limit_bal'].describe()

In [ ]:
df_t = df.copy()

In [ ]:
df_pay = pd.DataFrame(columns=['credit_assigned', 'total_bill','average_bill', 'total_payment', \
                               'average_payment', 'amt_owed','amt_owed_more_thn_credit','default_status',
                              'adv_pay', 'min_pay', 'delay_pay'])
df_pay['credit_assigned'] = df['limit_bal']
df_pay['total_bill'] = df[bill_amt_columns].sum(axis=1)
df_pay['average_bill'] = df[bill_amt_columns].mean(axis=1)
df_pay['total_payment'] = df[pay_amt_columns].sum(axis=1)
df_pay['average_payment'] = df[pay_amt_columns].mean(axis=1)
df_pay['amt_owed'] = df_pay['total_bill'] - df_pay['total_payment']


df_pay['amt_owed_more_thn_credit'] = np.where(
    (df_pay['credit_assigned'] - df_pay['amt_owed']) > 0,\
    0, 1)

df_5 = df[pay_status_columns]
df_pay['adv_pay'] = df_5[df_5 < 0 ].count(axis=1)
df_pay['min_pay'] = df_5[df_5 == 0 ].count(axis=1)
df_pay['delay_pay'] = df_5[df_5 > 0 ].count(axis=1)
df_pay['default_status'] = df['default_pay']
display(df_pay.head(10))

Exrracting more information from the dataset for new feature creation 
- _adv pay_ : Number of times -2,-1 has occured in pay status
- _min Pay_ : Number of times 0 has occured in pay status
- _delay pay_: Number of times 1 and above digits occured in pay status

In [ ]:
fig, ax = plt.subplots(2,3)
fig.set_size_inches(15,5)
fig.suptitle('Distribution of dalays in the past 6 months')

for i in range(len(pay_status_columns)):
    row,col = int(i/3), i%3

    d  = df[pay_status_columns[i]].value_counts()
    ax[row,col].bar(d.index, d, align='center', color='g')
    ax[row,col].set_title(pay_status_columns[i])

plt.tight_layout(pad=3.0, w_pad=0.5, h_pad=1.0)
plt.show()

In [ ]:
fig, ax = plt.subplots(2,3)
fig.set_size_inches(18,9)
fig.suptitle('Distribution of dalays repayments in the past 6 months')
for i in range(len(pay_status_columns)):
    row,col = int(i/3), i%3
    sns.countplot(x=pay_status_columns[i], hue="default_pay", data=df, ax=ax[row,col])
    ax[row,col].set_title(pay_status_columns[i])

plt.tight_layout(pad=3.0, w_pad=0.5, h_pad=1.0)
plt.show()

In [ ]:
cols_to_drop = ['pay_1','pay_2','pay_3','pay_4','pay_5','pay_6','sex','education','marriage','default_pay']
df_tmp = df.drop(labels=cols_to_drop,axis=1)
df_tmp.describe()


In [ ]:
lst = list(pay_amt_columns)
lst.extend(bill_amt_columns)
lst.extend(['limit_bal','age','default_pay'])

In [ ]:
df_corr = df.copy()
df_corr['default_pay'] = df_original['default payment next month']
#correlation matrix
cm = df_corr.corr()
plt.figure(figsize=(20,10))
sns.heatmap(cm, annot=True, cmap = 'viridis')

- __Limit Balance__ is negatively correlated with target variable.
- __bill amt__ features are internally strongly positively correlated

In [ ]:
print 'Marriage:  ', pd.unique(df['marriage'])
print 'Education: ', pd.unique(df['education'])
print 'Sex:       ', pd.unique(df['sex'])

As per documentation of the dataset, for education, we only have following<br/>
1 = graduate school; 2 = university; 3 = high school; 4 = others<br/>
We will replace 0,5,6 with 'others'

### Exploring Categorical Variables

In [ ]:
df['sex'] = df['sex'].astype('category').cat.rename_categories(['M', 'F'])
df['marriage'] = df['marriage'].astype('category').cat.rename_categories(['na','married', 'single', 'other'])
df['education'] = df['education'].astype('category').cat.rename_categories(['na','grad_school','university', 'high_school', 'others','unknown1','unknown2'])
df['age_cat'] = pd.cut(df['age'], range(0, 100, 10), right=False)
for i in pay_status_columns:
    df[i] = df[i].astype('category')

In [ ]:
fig, ax = plt.subplots(1,4)
fig.set_size_inches(16,4)
fig.suptitle('Defaulting by absolute numbers, for various demographics')

d = df.groupby(['default_pay', 'sex']).size()
p = d.unstack(level=1).plot(kind='bar', ax=ax[0])

d = df.groupby(['default_pay', 'marriage']).size()
p = d.unstack(level=1).plot(kind='bar', ax=ax[1])

d = df.groupby(['default_pay', 'education']).size()
p = d.unstack(level=1).plot(kind='bar', ax=ax[2])

d = df.groupby(['default_pay', 'age_cat']).size()
p = d.unstack(level=1).plot(kind='bar', ax=ax[3])

In [ ]:
fig, ax = plt.subplots(1,4)
fig.set_size_inches(16,4)
fig.suptitle('Defaulting by relative numbers given each class, for various demographics')

d = df.groupby(['default_pay', 'sex']).size().unstack(level=1)
d = d / d.sum()
p = d.plot(kind='bar', ax=ax[0])

d = df.groupby(['default_pay', 'marriage']).size().unstack(level=1)
d = d / d.sum()
p = d.plot(kind='bar', ax=ax[1])

d = df.groupby(['default_pay', 'education']).size().unstack(level=1)
d = d / d.sum()
p = d.plot(kind='bar', ax=ax[2])

d = df.groupby(['default_pay', 'age_cat']).size().unstack(level=1)
d = d / d.sum()
p = d.plot(kind='bar', ax=ax[3])

As we can see from the above graph, 
+ Distribution of Gender: 
    - Female credit card holders are more compared to Male
    - Female defaulters are more compared to Male
+ Distribution of Marriage: 
    - Single are holding more credit cards compared to Married and Others
    - Within defaulters Marital status with Others is very minimal. Married and Single have same number of defaulters
+ Distribution of Education:
    - University education has more credit crds. People with education values as Graduate and University have more credit cards.   
+ Distribution by Age:
    - Age 30 to 40 have more credit cards followed by Age 20 to 30.
    - Age 60 to 70 and Age 70 to 80 have very less credit card holders
    - Age 20 to 30 and 30 to 40 have more defaulters

In [ ]:
# Credit limit distribution
sns.distplot(df['limit_bal'],kde=True)
print 'Skewness on limit_bal is : ',skew(df['limit_bal'])

In [ ]:
#checking outliers for bill amount[1 to 6]
number_of_columns=6
number_of_rows = len(bill_amt_columns)-1/number_of_columns
plt.figure(figsize=(2*number_of_columns,5*number_of_rows))
for i in range(0,len(bill_amt_columns)):
    plt.subplot(number_of_rows + 1,number_of_columns,i+1)
    sns.set_style('whitegrid')
    sns.boxplot(df[bill_amt_columns[i]],color='green',orient='v')
    plt.tight_layout()

In [ ]:
#checking outliers for pay amount[1 to 6]
number_of_columns=6
number_of_rows = len(pay_amt_columns)-1/number_of_columns
plt.figure(figsize=(2*number_of_columns,5*number_of_rows))
for i in range(0,len(pay_amt_columns)):
    plt.subplot(number_of_rows + 1,number_of_columns,i+1)
    sns.set_style('whitegrid')
    sns.boxplot(df[pay_amt_columns[i]],color='green',orient='v')
    plt.tight_layout()

- As it can be observed that data is infested with outliers(by the logic of extreme values beyond 1.5*IQR), but since removing them would cost lot of valuable loss of information.We decided not to treat them as ouliers. 

***

# Feature Engineering

In [ ]:
#Checking missing values
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')
#Plane box indicates no missing values

- There are no missing values in data, its a clean dataset.

### Label Encoding

In [ ]:
df['grad_school'] = (df['education']==1).astype('int')
df['university'] = (df['education']==2).astype('int')
df['high_school']= (df['education']==3).astype('int')
df['others']= (df['education']==4).astype('int')
df['others']= (df['education']==5).astype('int')
df['others']= (df['education']==6).astype('int')
df['others']= (df['education']==0).astype('int')
df.drop('education',axis=1,inplace=True)

In [ ]:
df['male'] = (df['sex']=='M').astype('int')
df.drop('sex',axis=1,inplace=True)


df['married'] = (df['marriage']==1).astype('int')
df['single'] = (df['marriage']==2).astype('int')
df['na'] = (df['marriage']==3).astype('int')
df['na'] = (df['marriage']==0).astype('int')
df.drop(['marriage','age_cat'],axis=1,inplace=True)

# Since 0 is labeled as 'pay duly', every negative value should be seen as 0.
for i in pay_status_columns:
    df[i].replace({-2:-1,-1:-1},inplace=True)
 
df.head()

## Standardization

In [ ]:
#Scaling
scale = ['limit_bal','bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6','pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']
robust_scaler = RobustScaler()
df.loc[:,scale] = robust_scaler.fit_transform(df.loc[:,scale])

# df.head()

In [ ]:
col_values = df.columns.values
print type(col_values)
out = np.delete(col_values, np.where(col_values == 'default'))
col_values = out 
print col_values

# Feature Selection and Model Building

- We tried different methods with aim to find improvements in prediction outcomes.

### _Method 1_  : RFE - wrapper method

In [ ]:
lr = LogisticRegression(random_state=9)
dt = DecisionTreeClassifier(random_state=9)
rf = RandomForestClassifier(random_state=9)
X = df.drop('default_pay',axis=1)
y = df['default_pay']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=9,test_size=.2)
# create the RFE model and select 10 attributes
rfe1 = RFE(lr, 10)
rfe1 = rfe1.fit(X, y)
print ('Logistic Regression Feature Rankings:')
print(rfe1.ranking_)
print ('==')*40

rfe2 = RFE(dt, 10)
rfe2 = rfe2.fit(X, y)
print ('Decision Tree Feature Rankings:')
print(rfe2.ranking_)
print ('==')*40

rfe3 = RFE(rf, 10)
rfe3 = rfe3.fit(X, y)
print ('Random Forest Feature Rankings:')
print(rfe2.ranking_)
print ('==')*40

In [ ]:
# Test options and evaluation metric
seed = 7
scoring = 'accuracy'
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('RF2', RandomForestClassifier()))
models.append(('DT', DecisionTreeClassifier()))

from sklearn.model_selection import cross_val_score,KFold
# evaluate each model in turn
results = []
names = []

for name, model in models:
    kfold = KFold(n_splits=10, random_state=seed)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f" % (name, cv_results.mean())
    print(msg)

Logisitics regression performed better in kfold 

In [ ]:
#Selecting rfe1 features since lr performed good in KFold
X = df.iloc[:,[0,2,3,4,6,10,8,9,14,24]]
y = df.default_pay
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=9,test_size=.2)

lr.fit(X_train,y_train)

In [ ]:
y_pred = lr.predict(X_test)
model = lr
model_name = 'ITR1: Logistic Regression (post RFE)'
model_description = 'ITR1: Logistic Regression (with RFE)'
df_metric = save_performance_metrics(model, model_name, model_description, X_test, y_test, X_train, y_train)
# display(CMatrix(CM))
# display(df_metric)
# plot_roc_and_precision(model_name, model, X_test, y_test)

In [ ]:
print(classification_report(y_true=y_test,y_pred=y_pred))
print 'mcc : {}'.format( matthews_corrcoef(y_true=y_test,y_pred=y_pred))
tn, fp, fn, tp = confusion_matrix(y_true=y_test,y_pred=y_pred).ravel()
print("\n TN: {} FP: {} FN: {} TP: {} \n".format(tn, fp, fn, tp))
print 'auc: {}'.format(roc_auc_score(y_true=y_test,y_score=y_pred))
print 'accuracy: {}'.format(accuracy_score(y_true=y_test,y_pred=y_pred))

In [ ]:
#RFE with RandomForest since there was just .01 difference between lr
# create the RFE model and select 10 attributes
X = df.iloc[:,[0,1,2,8,9,10,11,12,13,15]]
y = df.default_pay
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=9,test_size=.2)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_true=y_test,y_pred=y_pred))
print 'mcc : {}'.format( matthews_corrcoef(y_true=y_test,y_pred=y_pred))
tn, fp, fn, tp = confusion_matrix(y_true=y_test,y_pred=y_pred).ravel()
print("\n TN: {} FP: {} FN: {} TP: {} \n".format(tn, fp, fn, tp))
print 'auc: {}'.format(roc_auc_score(y_true=y_test,y_score=y_pred))
print 'accuracy: {}'.format(accuracy_score(y_true=y_test,y_pred=y_pred))

In [ ]:
#Stcaking Ensemble with RFE1
X = df.iloc[:,[0,2,3,4,6,10,8,9,14,24]]
y = df['default_pay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=9)
X_train1, X_test1, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.2, random_state=9)

M1 = LogisticRegression()
M2 = DecisionTreeClassifier(max_depth=1,criterion='entropy')
M3 = DecisionTreeClassifier(max_depth=2,criterion='entropy')
M4 = DecisionTreeClassifier(max_depth=1,criterion='entropy')
M5 = DecisionTreeClassifier(max_depth=1,criterion='entropy')



M1.fit(X_train1,y_train2)
M2.fit(X_train1,y_train2)
M3.fit(X_train1,y_train2)
M4.fit(X_train1,y_train2)
M5.fit(X_train1,y_train2)


pred1 = M1.predict_proba(X_test1)
pred2 = M2.predict_proba(X_test1)
pred3 = M3.predict_proba(X_test1)
pred4 = M4.predict_proba(X_test1)
pred5 = M5.predict_proba(X_test1)


A = pd.DataFrame(pred1)[1]
B = pd.DataFrame(pred2)[1]
C = pd.DataFrame(pred3)[1]
D = pd.DataFrame(pred4)[1]
E = pd.DataFrame(pred5)[1]

X_meta = pd.concat([A,B,C,D,E],axis=1)

meta_classifier = RandomForestClassifier(max_depth=2,n_estimators=100)
meta_classifier.fit(X_meta,y_test2)


pred6 = M1.predict_proba(X_test)
pred7 = M2.predict_proba(X_test)
pred8 = M3.predict_proba(X_test)
pred9 = M4.predict_proba(X_test)
pred10 = M5.predict_proba(X_test)


F = pd.DataFrame(pred6)[1]
G = pd.DataFrame(pred7)[1]
H = pd.DataFrame(pred8)[1]
I = pd.DataFrame(pred9)[1]
J = pd.DataFrame(pred10)[1]

K = pd.concat([F,G,H,I,J], axis=1)
y_meta = meta_classifier.predict(K)

In [ ]:
print classification_report(y_meta,y_test)
print "auc:{}".format(roc_auc_score(y_meta,y_test)) 
print "mcc: {}".format(matthews_corrcoef(y_meta,y_test))
tn, fp, fn, tp = confusion_matrix(y_meta,y_test).ravel()
print ("\n TN: {} FP: {} FN: {} TP: {} \n".format(tn, fp, fn, tp))
print "accuracy: {}".format(accuracy_score(y_meta,y_test))

In [ ]:
#Stcaking Ensemble with RFE3
X = df.iloc[:,[0,2,1,10,8,9,11,12,13,15]]
y = df['default_pay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=9)
X_train1, X_test1, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.2, random_state=9)

M1 = LogisticRegression()
M2 = DecisionTreeClassifier(max_depth=1,criterion='entropy')
M3 = DecisionTreeClassifier(max_depth=2,criterion='entropy')
M4 = DecisionTreeClassifier(max_depth=1,criterion='entropy')
M5 = DecisionTreeClassifier(max_depth=1,criterion='entropy')



M1.fit(X_train1,y_train2)
M2.fit(X_train1,y_train2)
M3.fit(X_train1,y_train2)
M4.fit(X_train1,y_train2)
M5.fit(X_train1,y_train2)


pred1 = M1.predict_proba(X_test1)
pred2 = M2.predict_proba(X_test1)
pred3 = M3.predict_proba(X_test1)
pred4 = M4.predict_proba(X_test1)
pred5 = M5.predict_proba(X_test1)


A = pd.DataFrame(pred1)[1]
B = pd.DataFrame(pred2)[1]
C = pd.DataFrame(pred3)[1]
D = pd.DataFrame(pred4)[1]
E = pd.DataFrame(pred5)[1]

X_meta = pd.concat([A,B,C,D,E],axis=1)

meta_classifier = RandomForestClassifier(max_depth=2,n_estimators=100)
meta_classifier.fit(X_meta,y_test2)


pred6 = M1.predict_proba(X_test)
pred7 = M2.predict_proba(X_test)
pred8 = M3.predict_proba(X_test)
pred9 = M4.predict_proba(X_test)
pred10 = M5.predict_proba(X_test)


F = pd.DataFrame(pred6)[1]
G = pd.DataFrame(pred7)[1]
H = pd.DataFrame(pred8)[1]
I = pd.DataFrame(pred9)[1]
J = pd.DataFrame(pred10)[1]

K = pd.concat([F,G,H,I,J], axis=1)
y_meta = meta_classifier.predict(K)

In [ ]:
print classification_report(y_meta,y_test)
print "auc:{}".format(roc_auc_score(y_meta,y_test)) 
print "mcc: {}".format(matthews_corrcoef(y_meta,y_test))
tn, fp, fn, tp = confusion_matrix(y_meta,y_test).ravel()
print ("\n TN: {} FP: {} FN: {} TP: {} \n".format(tn, fp, fn, tp))
print "accuracy: {}".format(accuracy_score(y_meta,y_test))

## _Method 2: PCA_

In [ ]:
from sklearn.preprocessing import scale
from sklearn.feature_selection import chi2,f_classif,SelectKBest
from sklearn.decomposition import PCA
#converting dataframe into numpy Array
X = df.drop('default_pay',axis=1)
Y = df.default_pay

# Split-out validation dataset
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [ ]:
#Scaling the values
X_t = scale(X_train)

#initially lets create 28 components which is actual number of Variables we have
pca = PCA(n_components=28)

pca.fit(X_t)

#The amount of variance that each PC explains
var= pca.explained_variance_ratio_

#Cumulative Variance explains
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
#lets see Cumulative Variance plot
plt.plot(var1)

In [ ]:
#Looking at above plot I'm taking 16 variables
pca = PCA(n_components=16)
pca.fit(X_t)
X_train_PC=pca.fit_transform(X_t)
# X_train_PC

In [ ]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(random_state=9)))
models.append(('RF2', RandomForestClassifier(random_state=9)))
models.append(('DT', DecisionTreeClassifier(random_state=9)))

from sklearn.model_selection import cross_val_score,KFold
# evaluate each model in turn
results = []
names = []
# Test options and evaluation metric
seed = 7
scoring = 'accuracy'

for name, model in models:
    kfold = KFold(n_splits=10, random_state=seed)
    cv_results = cross_val_score(model, X_train_PC, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f" % (name, cv_results.mean())
    print(msg)

In [ ]:
# Comparing Algorithms
fig = plt.figure(figsize=(5,2))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)

#Scaling the X_validation data
X_v = scale(X_validation)

pca.fit(X_v)
X_validation_PC=pca.fit_transform(X_v)

In [ ]:
# Make predictions on validation dataset by choosing best Algorithm
lr = LogisticRegression(random_state=9)
lr.fit(X_train_PC, Y_train)
predictions = lr.predict(X_validation_PC)
print "accuracy: {}".format(accuracy_score(Y_validation, predictions))
tn, fp, fn, tp = confusion_matrix(Y_validation, predictions).ravel()
print ("\n TN: {} FP: {} FN: {} TP: {} \n".format(tn, fp, fn, tp))
print(classification_report(Y_validation, predictions))
print "mcc: {}".format(matthews_corrcoef(Y_validation, predictions))
print 'auc: {}'.format(roc_auc_score(Y_validation, predictions))

### _Method  3 : Select From Model_

In [ ]:
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE

X = df.drop('default_pay',axis=1)
y = df.default_pay
print('Before');
print(X.shape)

clf = RandomForestClassifier()
clf.fit(X, y)

print('\nFeature Importance');
print(clf.feature_importances_)

model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)

print('\nAfter'); 
print(X_new.shape)

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.20, random_state=42)
smote = SMOTE(random_state=9, kind="borderline2")               # With OverSampling Technique considering data is imbalance for a moment 
X_sample, y_sample = smote.fit_sample(X_train, y_train)
clf.fit(X_sample, y_sample)
y_pred = clf.predict(X_test)
print(classification_report(y_true=y_test,y_pred=y_pred))
print 'mcc : {}'.format( matthews_corrcoef(y_true=y_test,y_pred=y_pred))
tn, fp, fn, tp = confusion_matrix(y_true=y_test,y_pred=y_pred).ravel()
print("\n TN: {} FP: {} FN: {} TP: {} \n".format(tn, fp, fn, tp))
print 'auc: {}'.format(roc_auc_score(y_true=y_test,y_score=y_pred))
print 'accuracy: {}'.format(accuracy_score(y_true=y_test,y_pred=y_pred))

***

### _Method 4: SelectKBest_

In [ ]:
LogR = LogisticRegression(random_state=10)
LogR.fit(X_train, y_train)
y_pred_test = LogR.predict(X_test)
metrics.loc['accuracy','LogR'] = accuracy_score(y_pred=y_pred_test,y_true=y_test)
metrics.loc['precision','LogR'] = precision_score(y_pred=y_pred_test,y_true=y_test)
metrics.loc['recall','LogR'] = recall_score(y_pred=y_pred_test,y_true=y_test)
metrics.loc['auc_roc_score','LogR'] = roc_auc_score(y_score=y_pred_test,y_true=y_test)
CM = confusion_matrix(y_pred=y_pred_test,y_true=y_test)
model = LogR
model_name = 'ITR2(KBest,K=25): Simple Logistic Regression'
model_description = 'ITR2(KBest,K=25): basic Logistic Regression applied'
df_metric = save_performance_metrics(model, model_name, model_description, X_test, y_test, X_train, y_train)
display(CMatrix(CM))
display(df_metric)
plot_roc_and_precision(model_name, model, X_test, y_test)
# display(df_all_models)

In [ ]:
DTree = DecisionTreeClassifier(max_leaf_nodes=10,min_samples_split=30)
DTree.fit(X_train, y_train)
# print len(X_train[0])
y_pred_test = DTree.predict(X_test)
metrics.loc['accuracy','DTree'] = accuracy_score(y_pred=y_pred_test,y_true=y_test)
metrics.loc['precision','DTree'] = precision_score(y_pred=y_pred_test,y_true=y_test)
metrics.loc['recall','DTree'] = recall_score(y_pred=y_pred_test,y_true=y_test)
metrics.loc['auc_roc_score','DTree'] = roc_auc_score(y_score=y_pred_test,y_true=y_test)
CM = confusion_matrix(y_pred=y_pred_test,y_true=y_test)
model = DTree
model_name = 'ITR2(KBest,K=25): Decision Tree'
model_description = 'ITR2(KBest,K=25): Decision Tree (with max_leaf_node=10, sample_split=30)'
df_metric = save_performance_metrics(model, model_name, model_description, X_test, y_test, X_train, y_train)
display(CMatrix(CM))
display(df_metric)
plot_roc_and_precision(model_name, model, X_test, y_test)

***

### _Method 5: XgBoost_

In [ ]:
from xgboost import XGBClassifier

In [ ]:
#converting dataframe into numpy Array
X = df.drop('default_pay',axis=1)
Y = df.default_pay

# Split-out validation dataset
validation_size = 0.20
seed = 7
X_trainB, X_testB, Y_trainB, Y_testB = train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [ ]:
# fit model on training data
XGBoost = XGBClassifier()
XGBoost.fit(X_trainB, Y_trainB)

In [ ]:
# make predictions for test data
y_predB = XGBoost.predict(X_testB)
predictions = [round(value) for value in y_predB]

In [ ]:
# evaluate predictions
print(classification_report(y_true=Y_testB,y_pred=y_predB))
print 'mcc : {}'.format( matthews_corrcoef(y_true=Y_testB,y_pred=y_predB))
tn, fp, fn, tp = confusion_matrix(y_true=Y_testB,y_pred=y_predB).ravel()
print("\n TN: {} FP: {} FN: {} TP: {} \n".format(tn, fp, fn, tp))
print 'auc: {}'.format(roc_auc_score(y_true=Y_testB,y_score=y_predB))
print 'accuracy: {}'.format(accuracy_score(y_true=Y_testB,y_pred=y_predB))

In [ ]:
# plot feature importance using built-in function
from xgboost import plot_importance
plot_importance(XGBoost)
plt.show()

***

## Other approach - Month-wise models

In [ ]:
df1 = df.copy()
pd.unique(df_sample['pay_1'])

In [ ]:
for i in pay_status_columns:
    df_sample[i].replace({-2:0,-1:0,2:1,3:1,4:1,5:1,6:1,7:1,8:1,9:1},inplace=True)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, y,random_state = 9,train_size = 0.80)

In [ ]:
clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=4)
clf_gini.fit(X_train,y_train)

y_pred = clf_gini.predict(X_test)
y_pred

In [ ]:
auc = metrics.roc_auc_score(y_test, y_pred)
auc

In [ ]:
z = confusion_matrix(y_test, y_pred)
y = classification_report(y_test, y_pred)
train_score, test_score = clf_gini.score(X_train,y_train), clf_gini.score(X_test, y_test)
print train_score, test_score
print y, z
#print roc_auc_score(y_true=y_10,y_score=y_pred_final)

In [ ]:
df_sample_90, df_sample_10 = train_test_split(df,random_state = 9,train_size = 0.90)
y_train, y_test = train_test_split(df_sample_90['default_pay'],random_state = 9,train_size = 0.80)
y_10 = df_sample_10['default_pay']

In [ ]:
df_90 = df_sample_90
df_10 = df_sample_10
y_test.shape

In [ ]:
X1=df_90[['limit_bal','education','age','sex','bill_amt1','pay_amt1']]
y1=df_90['pay_1']
X2=df_90[['limit_bal','education','age','sex','bill_amt2','pay_amt2']]
y2=df_90['pay_2']
X3=df_90[['limit_bal','education','age','sex','bill_amt3','pay_amt3']]
y3=df_90['pay_3']
X4=df_90[['limit_bal','education','age','sex','bill_amt4','pay_amt4']]
y4=df_90['pay_4']
X5=df_90[['limit_bal','education','age','sex','bill_amt5','pay_amt5']]
y5=df_90['pay_5']
X6=df_90[['limit_bal','education','age','sex','bill_amt6','pay_amt6']]
y6=df_90['pay_6']
X1.info()

In [ ]:
rfe = RFE(X2_dt)
rfe = rfe.fit(X2, y2)
print(rfe.ranking_)

In [ ]:
X1=df_90[['limit_bal','bill_amt1','pay_amt1']]
y1=df_90['pay_1']
X2=df_90[['limit_bal','bill_amt2','pay_amt2']]
y2=df_90['pay_2']
X3=df_90[['limit_bal','bill_amt3','pay_amt3']]
y3=df_90['pay_3']
X4=df_90[['limit_bal','bill_amt4','pay_amt4']]
y4=df_90['pay_4']
X5=df_90[['limit_bal','bill_amt5','pay_amt5']]
y5=df_90['pay_5']
X6=df_90[['limit_bal','bill_amt6','pay_amt6']]
y6=df_90['pay_6']

In [ ]:
X1_train,X1_test,X2_train,X2_test,X3_train,X3_test,X4_train,X4_test,X5_train,X5_test,X6_train,X6_test,y1_train,y1_test,y2_train,y2_test,y3_train,y3_test,y4_train,y4_test,y5_train,y5_test,y6_train,y6_test = train_test_split(X1,X2,X3,X4,X5,X6,y1,y2,y3,y4,y5,y6,random_state = 9,train_size = 0.80)

In [ ]:
from sklearn.model_selection import cross_val_score,KFold
X1_dt = DecisionTreeClassifier(criterion='gini',max_depth=4)
kfold = KFold(n_splits=5)
cv_results = cross_val_score(X1_dt, X1_train, y1_train, cv=kfold, scoring='accuracy')
print cv_results.mean()

In [ ]:
X1_dt.fit(X1_train,y1_train)

y_pred1 = X1_dt.predict(X1_test)
y_pred1
a = confusion_matrix(y1_test, y_pred1)
b = classification_report(y1_test, y_pred1)
train_score, test_score = X1_dt.score(X1_train, y1_train), X1_dt.score(X1_test, y1_test)
print b, train_score, test_score
print y_pred1

In [ ]:
kfold = KFold(n_splits=5)
X1_lr = LogisticRegression()

cv_results = cross_val_score(X1_lr, X1_train, y1_train, cv=kfold, scoring='accuracy')
print cv_results.mean()

In [ ]:
X1_lr.fit(X1_train,y1_train)

y_pred1 = X1_lr.predict(X1_test)
y_pred1
a = confusion_matrix(y1_test, y_pred1)
b = classification_report(y1_test, y_pred1)
train_score, test_score = X1_dt.score(X1_train, y1_train), X1_dt.score(X1_test, y1_test)
print b, train_score, test_score
print y_pred1

In [ ]:
kfold = KFold(n_splits=5)
X1_rf = RandomForestClassifier()

cv_results = cross_val_score(X1_rf, X1_train, y1_train, cv=kfold, scoring='accuracy')
print cv_results.mean()

In [ ]:
X1_rf.fit(X1_train,y1_train)

y_pred1 = X1_rf.predict(X1_test)
y_pred1
a = confusion_matrix(y1_test, y_pred1)
b = classification_report(y1_test, y_pred1)
train_score, test_score = X1_dt.score(X1_train, y1_train), X1_dt.score(X1_test, y1_test)
print b, train_score, test_score
print y_pred1

In [ ]:
from sklearn.model_selection import cross_val_score,KFold
X2_dt = DecisionTreeClassifier(criterion='gini',max_depth=4)
kfold = KFold(n_splits=5)
cv_results = cross_val_score(X2_dt, X2_train, y2_train, cv=kfold, scoring='accuracy')
print cv_results.mean()

In [ ]:
X2_dt.fit(X2_train,y2_train)

y_pred2 = X2_dt.predict(X2_test)
y_pred2
c = confusion_matrix(y2_test, y_pred1)
d = classification_report(y2_test, y_pred2)
train_score, test_score = X2_dt.score(X2_train, y2_train), X2_dt.score(X2_test, y2_test)
print d, train_score, test_score

In [ ]:
X3_dt = DecisionTreeClassifier(criterion='gini',max_depth=4)
kfold = KFold(n_splits=5)
cv_results = cross_val_score(X3_dt, X3_train, y3_train, cv=kfold, scoring='accuracy')
print cv_results.mean()

In [ ]:
X3_dt = DecisionTreeClassifier(criterion='gini',max_depth=4)
X3_dt.fit(X3_train,y3_train)

y_pred3 = X3_dt.predict(X3_test)
y_pred3
e = confusion_matrix(y3_test, y_pred3)
f = classification_report(y3_test, y_pred3)
train_score, test_score = X3_dt.score(X3_train, y3_train), X3_dt.score(X3_test, y3_test)
print f, train_score, test_score

In [ ]:
X4_dt = DecisionTreeClassifier(criterion='gini',max_depth=4)
kfold = KFold(n_splits=5)
cv_results = cross_val_score(X4_dt, X4_train, y4_train, cv=kfold, scoring='accuracy')
print cv_results.mean()

In [ ]:
#X4_dt = DecisionTreeClassifier(criterion='gini',max_depth=4)
X4_dt.fit(X4_train,y4_train)

y_pred4 = X4_dt.predict(X4_test)
y_pred4
g = confusion_matrix(y4_test, y_pred4)
h = classification_report(y4_test, y_pred4)
train_score, test_score = X4_dt.score(X4_train, y4_train), X4_dt.score(X4_test, y4_test)
print h, train_score, test_score

In [ ]:
X5_dt = DecisionTreeClassifier(criterion='gini',max_depth=4)
kfold = KFold(n_splits=5)
cv_results = cross_val_score(X5_dt, X5_train, y5_train, cv=kfold, scoring='accuracy')
print cv_results.mean()

In [ ]:
#X5_dt = DecisionTreeClassifier(criterion='gini',max_depth=4)
X5_dt.fit(X5_train,y5_train)

y_pred5 = X5_dt.predict(X5_test)
y_pred5
i = confusion_matrix(y5_test, y_pred5)
j = classification_report(y5_test, y_pred5)
train_score, test_score = X5_dt.score(X5_train, y5_train), X5_dt.score(X5_test, y5_test)
print j, train_score, test_score

In [ ]:
X6_dt = DecisionTreeClassifier(criterion='gini',max_depth=4)
kfold = KFold(n_splits=5)
cv_results = cross_val_score(X6_dt, X6_train, y6_train, cv=kfold, scoring='accuracy')
print cv_results.mean()

In [ ]:
#X6_dt = DecisionTreeClassifier(criterion='gini',max_depth=4)
X6_dt.fit(X6_train,y6_train)

y_pred6 = X6_dt.predict(X6_test)
y_pred6
k = confusion_matrix(y6_test, y_pred6)
l = classification_report(y6_test, y_pred6)
train_score, test_score = X6_dt.score(X6_train, y6_train), X6_dt.score(X6_test, y6_test)
print l, train_score, test_score

In [ ]:
y_pred = pd.DataFrame(list(zip(y_pred1,y_pred2,y_pred3,y_pred4,y_pred5,y_pred6)),columns=['p1','p2','p3','p4','p5','p6'])
y_pred.head()
y_test.reshape(-1,1)
lr = LogisticRegression()
lr.fit(y_pred,y_test.reshape(-1,1))

In [ ]:
X1_10=df_10[['limit_bal','bill_amt1','pay_amt1']]
y1_10=df_10['pay_1']
X2_10=df_10[['limit_bal','bill_amt2','pay_amt2']]
y2_10=df_10['pay_2']
X3_10=df_10[['limit_bal','bill_amt3','pay_amt3']]
y3_10=df_10['pay_3']
X4_10=df_10[['limit_bal','bill_amt4','pay_amt4']]
y4_10=df_10['pay_4']
X5_10=df_10[['limit_bal','bill_amt5','pay_amt5']]
y5_10=df_10['pay_5']
X6_10=df_10[['limit_bal','bill_amt6','pay_amt6']]
y6_10=df_10['pay_6']

In [ ]:
y_10_pred1 = X1_dt.predict(X1_10)
y_10_pred1

In [ ]:
y_10_pred2 = X2_dt.predict(X2_10)
y_10_pred2

In [ ]:
y_10_pred3 = X3_dt.predict(X3_10)
y_10_pred3

In [ ]:
y_10_pred4 = X4_dt.predict(X4_10)
y_10_pred4

In [ ]:
y_10_pred5 = X5_dt.predict(X5_10)
y_10_pred5

In [ ]:
y_10_pred6 = X6_dt.predict(X6_10)
y_10_pred6

In [ ]:
y_pred_10 = pd.DataFrame(list(zip(y_10_pred1,y_10_pred2,y_10_pred3,y_10_pred4,y_10_pred5,y_10_pred6)),columns=['p10_1','p10_2','p10_3','p10_4','p10_5','p10_6'])
y_pred_10.head()

In [ ]:
y_pred_final.reshape(-1,1).shape

In [ ]:
y_pred_final = lr.predict(y_pred_10)
y_pred_final

In [ ]:
y_10.reshape(-1,1)

In [ ]:
m = confusion_matrix(y_10, y_pred_final)
n = classification_report(y_10, y_pred_final)
train_score, test_score = lr.score(y_pred,y_test), lr.score(y_pred_10, y_10)
print train_score, test_score
print n
print roc_auc_score(y_true=y_10,y_score=y_pred_final)

***